In [ ]:
import time
import torch
import pandas as pd
import numpy as np
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer

print("Torch version :", torch.__version__)
print("Pandas version :", pd.__version__)
print("NumPy version :", np.__version__)
print("TFT installé correctement !")
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

In [ ]:
# Charger les données MetroPT-3
df = pd.read_csv(r"..\..\..\..\Datasources\MetroPT3_new_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

In [ ]:
# Convertir la colonne 'timestamp' en type datetime

df['timestamp'] = pd.to_datetime(df['timestamp'])

# Définir les colonnes continues et catégoriques pour l'entrainement
continuous_features  = ["TP2","H1","DV_pressure", "Oil_temperature", "Motor_current"]
categorical_features = ["COMP", "DV_eletric", "Towers"]

# Convertir les colonnes catégoriques en type "category"
for col in categorical_features:
    df[col] = df[col].astype('category')

# Convertir la colonne 'panne' en type category
df['panne'] = df['panne'].astype('category')

# Liste des colonnes nécessaires
columns_to_keep = ["timestamp"] + continuous_features + ["panne"] + categorical_features

# Réduire le DataFrame
df = df[columns_to_keep]

# Afficher les premières lignes pour vérifier
display(df.head(2))

In [ ]:
# Dataset commence le 2020-04-12 11:20:00 et se termine le 2020-07-17 06:00:00
pannes = [
    {'id': 'Panne1',  'start': '2020-04-12 11:50:00', 'end': '2020-04-12 23:30:00'},
    {'id': 'Panne2',  'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'id': 'Panne3',  'start': '2020-04-19 00:00:00', 'end': '2020-04-19 01:30:00'},
    {'id': 'Panne4',  'start': '2020-04-29 03:20:00', 'end': '2020-04-29 04:00:00'},
    {'id': 'Panne5',  'start': '2020-04-29 22:00:00', 'end': '2020-04-29 22:20:00'},
    {'id': 'Panne6',  'start': '2020-05-13 14:00:00', 'end': '2020-05-13 23:59:00'},
    {'id': 'Panne7',  'start': '2020-05-18 05:00:00', 'end': '2020-05-18 05:30:00'},
    {'id': 'Panne8',  'start': '2020-05-19 10:10:00', 'end': '2020-05-19 11:00:00'},
    {'id': 'Panne9',  'start': '2020-05-19 22:10:00', 'end': '2020-05-19 23:59:00'},
    {'id': 'Panne10', 'start': '2020-05-20 00:00:00', 'end': '2020-05-20 20:00:00'},
    {'id': 'Panne11', 'start': '2020-05-23 09:50:00', 'end': '2020-05-23 10:10:00'},
    {'id': 'Panne12', 'start': '2020-05-29 23:30:00', 'end': '2020-05-29 23:59:00'},
    {'id': 'Panne13', 'start': '2020-05-30 00:00:00', 'end': '2020-05-30 06:00:00'},
    {'id': 'Panne14', 'start': '2020-06-01 15:00:00', 'end': '2020-06-01 15:40:00'},
    {'id': 'Panne15', 'start': '2020-06-03 10:00:00', 'end': '2020-06-03 11:00:00'},
    {'id': 'Panne16', 'start': '2020-06-05 10:00:00', 'end': '2020-06-05 23:59:00'},
    {'id': 'Panne17', 'start': '2020-06-06 00:00:00', 'end': '2020-06-06 23:59:00'},
    {'id': 'Panne18', 'start': '2020-06-07 00:00:00', 'end': '2020-06-07 14:30:00'},
    {'id': 'Panne19', 'start': '2020-07-08 17:30:00', 'end': '2020-07-08 19:00:00'},
    {'id': 'Panne20', 'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
    {'id': 'Panne21', 'start': '2020-07-17 04:30:00', 'end': '2020-07-17 05:30:00'}
         ]

In [ ]:
# Séparation Train/Test et Conversion de time_idx


# Définir la date de séparation entre Train et Test
split_date = "2020-06-05 23:59:10"  # Séparation temporelle (Fin de la panne 16)

# Séparer les données en train/test
df_train = df[df["timestamp"] < split_date].copy()
df_test = df[df["timestamp"] >= split_date].copy()

# Transformer la colonne timestamp en index numérique basé sur le temps écoulé
df_train["time_idx"] = (df_train["timestamp"] - df["timestamp"].min()).dt.total_seconds().astype(int)
df_test["time_idx"] = (df_test["timestamp"] - df["timestamp"].min()).dt.total_seconds().astype(int)

# Trier les données pour s'assurer de l'ordre temporel
df_train = df_train.sort_values(by=["COMP", "time_idx"]).reset_index(drop=True)
df_test = df_test.sort_values(by=["COMP", "time_idx"]).reset_index(drop=True)

print(f"Entraînement : {len(df_train)} lignes | Test : {len(df_test)} lignes")
print("Conversion de timestamp en index temporel terminée.")

In [ ]:
# Vérification des Groupes et des Données

# Vérifier que COMP est bien une catégorie
df_train["COMP"] = df_train["COMP"].astype(str).astype("category")
df_test["COMP"] = df_test["COMP"].astype(str).astype("category")

print(f"Groupes uniques dans df_train : {df_train['COMP'].unique()}")
print(f"Groupes uniques dans df_test : {df_test['COMP'].unique()}")

# Vérifier combien de timestamps uniques par groupe
grouped_counts_test = df_test.groupby(["COMP"])["time_idx"].nunique()
print("Timestamps uniques par groupe dans df_test :", grouped_counts_test)


In [ ]:
# Création des Encodeurs Catégoriques

# Convertir les colonnes catégoriques en chaînes de caractères pour PyTorch Forecasting
categorical_features = ["COMP", "DV_eletric", "Towers"]

for col in categorical_features:
    df_train[col] = df_train[col].astype(str).astype("category")
    df_test[col] = df_test[col].astype(str).astype("category")

    
print("Les colonnes catégoriques sont bien en chaînes de caractères.")
# Définir les encodeurs pour les variables catégoriques
categorical_encoders = {
    "COMP": NaNLabelEncoder(),
    "DV_eletric": NaNLabelEncoder(),
    "Towers": NaNLabelEncoder(),
}

print("Encodeurs catégoriques définis.")

In [ ]:
# Création des Datasets pour TFT

# Démarrer le chronometre
start_time = time.time()

# Définir l'horizon de prévision et la fenêtre d'observation
max_encoder_length = 5  # Réduction pour s'assurer d'avoir assez de séquences
max_prediction_length = 2 # Réduction pour éviter les suppressions de séries


df_train["COMP"] = df_train["COMP"].astype(str).astype("category")
df_train["Towers"] = df_train["Towers"].astype(str).astype("category")


# Vérifier si certaines séquences sont trop courtes pour être utilisées
for comp in df_test["COMP"].unique():
    subset = df_test[df_test["COMP"] == comp]
    unique_timestamps = subset["time_idx"].nunique()
    if unique_timestamps < (max_encoder_length + max_prediction_length):
        print(f"⚠ Série COMP={comp} est trop courte avec {unique_timestamps} points !")
    else:
        print(f"Série COMP={comp} est suffisante avec {unique_timestamps} points.")

      

# Création du dataset pour TFT (Train)
train_dataset = TimeSeriesDataSet(
    df_train,
    time_idx="time_idx",
    target="panne",
    group_ids=["COMP"],  
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=10,
    max_prediction_length=3,
    min_encoder_length=5,  
    min_prediction_length=2,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)

print(f"Dataset train créé avec {len(train_dataset)} séquences.")


# Création du dataset pour TFT (Test)
test_dataset = TimeSeriesDataSet(
    df_test,
    time_idx="time_idx",
    target="panne",
    group_ids=["COMP"],  
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=10,
    max_prediction_length=3,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)

# Vérifier les tailles des datasets
print(f"Train dataset: {len(train_dataset)} séquences | Test dataset: {len(test_dataset)} séquences")

# Arrêter le chronomètre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

In [ ]:
# Création des Datasets pour TFT

# Définir l'horizon de prévision et la fenêtre d'observation
max_encoder_length = 10  # Réduction pour s'assurer d'avoir assez de séquences
max_prediction_length = 3 # Réduction pour éviter les suppressions de séries


# Création du dataset pour TFT (Train)
train_dataset = TimeSeriesDataSet(
    df_train,
    time_idx="time_idx",
    target="panne",
    group_ids=["COMP"],  
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=10,
    max_prediction_length=3,
    min_encoder_length=5,  
    min_prediction_length=2,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)

print(f"✅ Dataset train créé avec {len(train_dataset)} séquences.")


In [ ]:
df_test["group_id"] = "compresseur"  # ✅ Définir un group_id unique

test_dataset = TimeSeriesDataSet(
    df_test,
    time_idx="time_idx",
    target="panne",
    group_ids=["group_id"],  # ✅ Modifier group_ids pour correspondre à l'approche du document
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=5,
    max_prediction_length=2,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)


In [ ]:
# Préparer les DataLoaders

# Démarrer le chronometre
start_time = time.time()

# Création des DataLoaders pour l'entraînement et le test
train_dataloader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=0
)

test_dataloader = DataLoader(
    test_dataset, batch_size=64, shuffle=False, num_workers=0
)

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

In [ ]:
# Définir et entraîner le modèle TFT

# Démarrer le chronometre
start_time = time.time()




# Définition du modèle TFT
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=0.01,
    hidden_size=16,
    attention_head_size=4,
    dropout=0.1,
    output_size=7,  # Nombre de quantiles
    loss=QuantileLoss(),
    logging_metrics=[],
)

# Configuration du Trainer avec EarlyStopping
trainer = Trainer(
    max_epochs=30,
    gradient_clip_val=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5)],
)

# Entraînement du modèle
trainer.fit(
    tft, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader
)

print(" Entraînement terminé.")



# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

In [ ]:
# Faire des prédictions et identifier les pannes imminentes

# Démarrer le chronometre
start_time = time.time()

# Générer des prédictions sur le jeu de test
raw_predictions, x = tft_loaded.predict(test_dataloader, return_x=True)

# Transformer les prédictions en DataFrame
pred_df = pd.DataFrame({
    "timestamp": x["encoder_time_idx"][:, -1].cpu().numpy(),
    "panne_predite": raw_predictions[:, 0].cpu().numpy(),
})

# Afficher les résultats des prédictions
print(pred_df.head(10))

# Visualisation des pannes anticipées sur le jeu de test
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(df_test["timestamp"], df_test["panne"], label="Réel", color="blue")
plt.plot(pred_df["timestamp"], pred_df["panne_predite"], label="Prédit", color="red")
plt.xlabel("Temps")
plt.ylabel("Panne (0 = normal, 2 = pré-panne, 1 = panne)")
plt.legend()
plt.show()


# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

In [ ]:
# Sauvegarde du modèle entrainé

# Démarrer le chronometre
start_time = time.time()

model_path = r"..\..\Generated_Files\TFT\tft_model.pth"  
torch.save(tft.state_dict(), model_path)
print(f"Modèle sauvegardé sous : {model_path}")

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

In [ ]:
# Recharger le modèle entrainé

# Démarrer le chronometre
start_time = time.time()

tft_loaded = TemporalFusionTransformer.from_dataset(train_dataset)

# Charger les poids du modèle sauvegardé
tft_loaded.load_state_dict(torch.load(r"..\..\Generated_Files\TFT\tft_model.pth"))
tft_loaded.eval()   # Mettre le modèle en mode évaluation

print("Modèle chargé avec succès !")

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

In [ ]:
***********************************************************************************
***********************************************************************************

In [1]:
import pandas as pd
import time
import torch
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import EarlyStopping

# Charger le dataset
df = pd.read_csv(r"..\..\..\..\Datasources\MetroPT3_new_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

# Convertir timestamp en datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Définir la date de séparation entre Train et Test
split_date = "2020-06-05 23:59:10"  # Séparation temporelle (Fin de la panne 16)

# Séparer les données en train/test
df_train = df[df["timestamp"] < split_date].copy()
df_test = df[df["timestamp"] >= split_date].copy()

# Recalculer time_idx en divisant par 10 pour éviter les grands nombres
df_train["time_idx"] = ((df_train["timestamp"] - df_train["timestamp"].min()).dt.total_seconds() // 10).astype(int)
df_test["time_idx"] = ((df_test["timestamp"] - df_test["timestamp"].min()).dt.total_seconds() // 10).astype(int)

print(f"Min time_idx in df_train: {df_train['time_idx'].min()}, Max: {df_train['time_idx'].max()}")
print(f"Min time_idx in df_test: {df_test['time_idx'].min()}, Max: {df_test['time_idx'].max()}")


Min time_idx in df_train: 0, Max: 471114
Min time_idx in df_test: 0, Max: 356405


In [2]:
# Définir les colonnes catégoriques et les forcer en str avant de les convertir en category
for col in ["COMP", "DV_eletric", "Towers"]:
    df_train[col] = df_train[col].astype(int).astype(str).astype("category")
    df_test[col] = df_test[col].astype(int).astype(str).astype("category")

# Conversion explicite de panne
df_train["panne"] = df_train["panne"].astype(int).astype(str).astype("category")
df_test["panne"] = df_test["panne"].astype(int).astype(str).astype("category")

print("Conversion des colonnes catégoriques et de panne terminée.")

Conversion des colonnes catégoriques et de panne terminée.


In [3]:
# Ajouter un group_id unique pour tout le dataset
df_train["group_id"] = "compresseur"
df_test["group_id"] = "compresseur"

print("group_id ajouté.")

group_id ajouté.


In [4]:
# Définir les encodeurs pour les variables catégoriques
categorical_encoders = {
    "COMP": NaNLabelEncoder(),
    "DV_eletric": NaNLabelEncoder(),
    "Towers": NaNLabelEncoder(),
}

print("Encodeurs catégoriques définis.")

Encodeurs catégoriques définis.


In [5]:
# Définir l'horizon de prévision et la fenêtre d'observation
max_encoder_length = 30  # Fenêtre plus longue pour éviter la suppression
max_prediction_length = 1  # Une seule prédiction pour éviter les erreurs

train_dataset = TimeSeriesDataSet(
    df_train,
    time_idx="time_idx",
    target="panne",
    group_ids=["COMP"],
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=10,
    max_prediction_length=3,
    min_encoder_length=5,
    min_prediction_length=2,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)

print(f"Dataset train créé avec {len(train_dataset)} séquences.")

Dataset train créé avec 476698 séquences.


In [7]:
test_dataset = TimeSeriesDataSet(
    df_test,
    time_idx="time_idx",
    target="panne",
    group_ids=["COMP"],
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=10,  # Doit être identique à train_dataset
    max_prediction_length=3,  # Doit être identique à train_dataset
    min_encoder_length=5,
    min_prediction_length=2,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)

print(f"Dataset test créé avec {len(test_dataset)} séquences.")

Dataset test créé avec 357868 séquences.


In [8]:
print(f"Nombre de groupes dans test_dataset: {len(test_dataset.index) if hasattr(test_dataset, 'index') else 'N/A'}")

Nombre de groupes dans test_dataset: 357868


In [9]:
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

# Définition du modèle TFT (Pas besoin de `LightningModule`)
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=1e-3,
    hidden_size=16,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Pas besoin d'assertion sur `LightningModule`
print(f"Modèle TFT initialisé avec {tft.size()} paramètres.")

C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


Modèle TFT initialisé avec 21007 paramètres.


C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\pytorch_forecasting\models\temporal_fusion_transformer\__init__.py:171: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default optimizer will be 'adam' regardless of whether pytorch_optimizer is installed, in order to minimize the number of dependencies in default parameter settings. Users who wish to ensure their code continues using 'ranger' as optimizer should ensure that pytorch_optimizer is installed, and set the optimizer parameter explicitly to 'ranger'.
  super().__init__(loss=loss, logging_metrics=logging_metrics, **kwargs)


In [10]:
from pytorch_lightning import LightningModule

# Création du wrapper LightningModule
class TFTLightningModule(LightningModule):
    def __init__(self, tft):
        super().__init__()
        self.model = tft

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.model(x)
        loss = self.model.loss(y_pred, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        if batch is None or not batch:
            print(f"❌ Batch {batch_idx} est VIDE dans validation_step()")
            return None  #  Évite une erreur bloquante

        # Vérifie la structure du batch
        if "x" not in batch or "y" not in batch:
            print(f"❌ Batch {batch_idx} mal structuré : {batch}")
            return None

        x = batch["x"]
        y = batch["y"]

        y_pred = self.model(x)
        loss = self.model.loss(y_pred, y)

        return {"val_loss": loss}

    def configure_optimizers(self):
        return self.model.configure_optimizers()

# Encapsulation du modèle TFT dans `LightningModule`
tft_lightning = TFTLightningModule(tft)

In [11]:
from pytorch_lightning.callbacks import EarlyStopping, TQDMProgressBar
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
import torch

def tft_collate_fn(batch):
    batch = [b for b in batch if b is not None]  # Supprime les batchs `None`

    if len(batch) == 0:
        print("⚠ Avertissement : Un batch vide a été détecté. Il sera remplacé par des zéros.")
        return {
            "x": {
                "x_cat": torch.zeros((1, 2), dtype=torch.long),  # Ajuste selon le nombre de variables catégoriques
                "x_cont": torch.zeros((1, 5), dtype=torch.float32)  # Ajuste selon le nombre de variables continues
            },
            "y": torch.zeros((1, 1), dtype=torch.float32)
        }

    try:
        inputs, targets = zip(*batch)  # Sépare les entrées (x) et les cibles (y)
        return {
            "x": default_collate(inputs),
            "y": default_collate(targets)
        }
    except Exception as e:
        print(f"Erreur dans `collate_fn`: {e}")
        return {
            "x": {
                "x_cat": torch.zeros((1, 2), dtype=torch.long),
                "x_cont": torch.zeros((1, 5), dtype=torch.float32)
            },
            "y": torch.zeros((1, 1), dtype=torch.float32)
        }

    
# Définition des DataLoaders
batch_size = 64
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True, collate_fn=tft_collate_fn
)
test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True, collate_fn=tft_collate_fn
)

for batch_idx, batch in enumerate(train_dataloader):
    if not batch:
        print(f"❌ Batch {batch_idx} est VIDE dans `train_dataloader`.")
    elif isinstance(batch, dict):
        print(f"✅ Batch {batch_idx} est un dictionnaire contenant {list(batch.keys())}")
    else:
        print(f"❌ Type inconnu pour Batch {batch_idx}: {type(batch)}")
    
    if batch_idx == 5:  # Vérifie seulement les 5 premiers batchs
        break

        
# Callback d'Early Stopping pour éviter le surajustement
early_stop_callback = EarlyStopping(monitor="val_loss", patience=5, verbose=True, mode="min")

# Callback de Progression avec `tqdm`
progress_bar = TQDMProgressBar(refresh_rate=10)  # Mettre à jour toutes les 10 itérations

# Définition de l'entraîneur
trainer = Trainer(
    max_epochs=30,
    accelerator="cpu",  # ✅ Forcer l'utilisation du CPU
    enable_progress_bar=True,
    callbacks=[early_stop_callback, progress_bar]  # ✅ Ajout de la `Progress Bar`
)

for batch_idx, batch in enumerate(train_dataloader):
    if not batch:
        print(f"❌ Batch {batch_idx} est VIDE dans `train_dataloader`.")
    elif isinstance(batch, dict):
        print(f"✅ Batch {batch_idx} est un dictionnaire contenant {list(batch.keys())} avec {batch['x']['x_cat'].shape[0]} échantillons")
    else:
        print(f"❌ Type inconnu pour Batch {batch_idx}: {type(batch)}")
    
    if batch_idx == 5:  # Vérifie seulement les 5 premiers batchs
        break

        
# Entraînement du modèle
trainer.fit(tft_lightning, train_dataloader, test_dataloader)

print("Entraînement terminé avec succès !")

Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [12, 2] at entry 1
✅ Batch 0 est un dictionnaire contenant ['x', 'y']
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [7, 2] at entry 6
✅ Batch 1 est un dictionnaire contenant ['x', 'y']
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [11, 2] at entry 5
✅ Batch 2 est un dictionnaire contenant ['x', 'y']
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [10, 2] at entry 0 and [13, 2] at entry 1
✅ Batch 3 est un dictionnaire contenant ['x', 'y']
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [8, 2] at entry 1
✅ Batch 4 est un dictionnaire contenant ['x', 'y']
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [12, 2] at entry 4
✅ Batch 5 est un dictionnai

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [12, 2] at entry 1
✅ Batch 0 est un dictionnaire contenant ['x', 'y'] avec 1 échantillons
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [10, 2] at entry 15
✅ Batch 1 est un dictionnaire contenant ['x', 'y'] avec 1 échantillons
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [12, 2] at entry 3
✅ Batch 2 est un dictionnaire contenant ['x', 'y'] avec 1 échantillons
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [12, 2] at entry 8
✅ Batch 3 est un dictionnaire contenant ['x', 'y'] avec 1 échantillons
Erreur dans `collate_fn`: stack expects each tensor to be equal size, but got [13, 2] at entry 0 and [11, 2] at entry 11
✅ Batch 4 est un dictionnaire contenant ['x', 'y'] avec 1 échantillons
Erreur dans `collate_fn`: stack expects eac


  | Name  | Type                      | Params | Mode 
------------------------------------------------------------
0 | model | TemporalFusionTransformer | 21.0 K | train
------------------------------------------------------------
21.0 K    Trainable params
0         Non-trainable params
21.0 K    Total params
0.084     Total estimated model params size (MB)
326       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Erreur dans `collate_fn`: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>


KeyError: 'encoder_lengths'

In [ ]:
import os

# Sauvegarde du modèle après l'entraînement
save_path = r"..\..\Generated_Files\TFT\tft_model_checkpoint.ckpt"
trainer.save_checkpoint(save_path)

print(f"Modèle sauvegardé sous : {save_path}")
